# Data export script

- @author Gong Zequn (zequn.gong@u.nus.edu)
- @create 2021.06.28
- @modify 2021.07.22

- @author Shashank Acharya (acharya.s@u.nus.edu)
- @modify 2022.03.20

In [ ]:
!pip install pymysql pandas

{'_db_host': 'mysql',
 '_db_port': 3306,
 '_db_name': 'singlishwords',
 '_db_charset': 'utf8',
 '_redis_host': 'redis',
 '_redis_port': 6379,
 '_db_user': 'csqsiew',
 '_db_passwd': 'u98x7v89asx'}

# Initialise Database Connection

Here, we create a class for the `libs.database` submodule to initialise the connections.

The config is the same across all notebooks in this folder.
- The host is `mysql`
- The port that `mysql` runs on is `3306`
- The port that the `redis` service runs on is `6739`
- If you are testing locally with the database setup in singlish-words-backend, replace the username and password with your own MySQL username and password.

In [ ]:
import json
cfg = json.load(open('config.json'))

class Configs:
    _db_host = cfg['_db_host']
    _db_port = cfg['_db_port']
    
    _db_name = cfg['_db_name']
    _db_charset = cfg['_db_charset']

    _redis_host = cfg['_redis_host']
    _redis_port = cfg['_redis_port']
    
    _db_user = cfg['_db_user']
    _db_passwd = cfg['_db_passwd']

This block will initialise the database connection.

In [ ]:
import libs.database as database

database.initConnections(Configs)

# Create Table Dictionaries

The functions used to retrieve data from the following tables are under `libs/database.py`

In [ ]:
questions = database.getQuestions()
answers = database.getAnswers()
respondents = database.getRespondents()

# Converting Dataframes to Tables using Pandas

First, we will import the `pandas` module

In [ ]:
import pandas as pd

Next, we will convert each of the table dictionaries to a Pandas dataframe.

In [ ]:
questions_df = pd.DataFrame.from_dict(questions.values())
answers_df = pd.DataFrame.from_dict(answers.values())
respondents_df = pd.DataFrame.from_dict(respondents.values())

As a checkpoint, in case we are unable to merge the dataframes later, let's first export the current dataframes to CSV as is.

In [ ]:
questions_df.to_csv('./export-data/questions.csv', index=False)
answers_df.to_csv('./export-data/answers.csv', index=False)
respondents_df.to_csv('./export-data/respondents.csv', index=False)

# Merging dataframes

## a. Question-Answer Merge

In [ ]:
qa = pd.merge(
    answers_df, 
    questions_df, 
    left_on='question_id', 
    right_on='id', 
    suffixes=['_answer', '_question']
)

## b. Question-Answer-Response Merge

In [ ]:
qar = pd.merge(
    qa,
    respondents_df, 
    left_on='respondent_id', 
    right_on='id', 
    suffixes=['_qa', '_respondent']
)

Cleaning the overlapping (extra) columns in the `qar` dataframe.

In [ ]:
qar = qar.drop(columns=['id_answer', 'id_question', 'id'])

# Export the merged CSV file

In [ ]:
qar.to_csv('./export-data/data.csv', index=False)

# Create the nested JSON format

First, we will import the `json` module

In [ ]:
import json

Then, for each respondent, we will set the results to an empty array that will contain the answers.

In [ ]:
results = {k: v for k, v in respondents.items()}
for k, v in results.items():
    results[k]['answers'] = []

For each of the respondents' answers, we will add the question details, associations entered, and time spent.

In [ ]:
for k, v in answers.items():
    results[v['respondent_id']]['answers'].append({
        'question': questions[v['question_id']]['word'],
        'associations': [v['association1'], v['association2'], v['association3']],
        'is_recognised_word': v['is_recognised_word'],
        'time_spend': v['time_spend'],
    })

We are now ready to export this to a JSON file.

In [ ]:
with open('./export-data/data.json', 'w') as f:
    json.dump(results, f)